In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import torch
import pandas as pd
import numpy as np
import lightning as L

## Data Module

In [3]:
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

from dataset.dataset import ForexDataset

class ForexDataModule(L.LightningDataModule):
    def __init__(
        self,
        data,
        IDs: list,
        sequence_length: int,
        horizon: int,
        features: list,
        target: list,
        batch_size: int = 64,
        num_workers: int = 0,
        val_split: float = 0.2,
        shuffle: bool = True,
        random_state: int = 42
    ):
        super().__init__()
        self.data = data
        self.IDs = IDs
        self.sequence_length = sequence_length
        self.horizon = horizon
        self.features = features
        self.target = target
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.val_split = val_split
        self.shuffle = shuffle
        self.random_state = random_state

    def setup(self, stage=None):

        train_idx, val_idx = train_test_split(
            self.IDs,
            test_size=self.val_split,
            shuffle=self.shuffle,
            random_state=self.random_state
        )

        self.train_dataset = ForexDataset(
            self.data, train_idx, self.sequence_length, self.horizon, self.features, self.target
        )

        self.val_dataset = ForexDataset(
            self.data, val_idx, self.sequence_length, self.horizon, self.features, self.target
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
            pin_memory=True
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=True
        )

    def test_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=True
        )


# GRU classification Model

We've tried different criterion.
    a. softmax + mse
    b. raw logits + cross entropy
We found that cross entropy performs better when using pytorch

In [4]:
import torch
from torch import nn
import lightning as L
from torchmetrics.classification import MulticlassAccuracy


class GRUModel(nn.Module):
    def __init__(self, n_features, output_size, n_hidden, n_layers, dropout):
        super().__init__()

        self.gru = nn.GRU(
            input_size=n_features,
            hidden_size=n_hidden,
            num_layers=n_layers,
            batch_first=True,
            dropout=dropout
        )
        self.linear = nn.Linear(n_hidden, output_size)
        self.softmax = nn.Softmax(1)
        

    def forward(self, x):
        self.gru.flatten_parameters()
        _, hidden = self.gru(x)
        logits = self.linear(hidden[-1])
        # return self.softmax(logits)
        return logits


class GRUModule(L.LightningModule):
    def __init__(self, n_features=1, output_size=1, n_hidden=64, n_layers=2, dropout=0.0):
        super().__init__()
        self.save_hyperparameters()

        self.model = GRUModel(
            n_features=self.hparams.n_features,
            output_size=self.hparams.output_size,
            n_hidden=self.hparams.n_hidden,
            n_layers=self.hparams.n_layers,
            dropout=self.hparams.dropout,
        )

        self.criterion = nn.CrossEntropyLoss()
        self.test_accuracy = MulticlassAccuracy(num_classes=output_size)

    def forward(self, x, labels=None):
        output = self.model(x)
        loss = 0
        if labels is not None:
            labels = labels.squeeze().long()
            loss = self.criterion(output, labels)
        return loss, output

    def training_step(self, batch, batch_idx):
        x, y, _ = batch
        loss, out = self(x, y)

        self.log('train_loss', loss, prog_bar=True, logger=True)
        return {
            'loss': loss
        }

    def validation_step(self, batch, batch_idx):
        x, y, _ = batch
        loss, out = self(x, y)

        self.log('val_loss', loss, prog_bar=True, logger=True)
        return {
            'loss': loss
        }

    def test_step(self, batch, batch_idx):
        x, y, _ = batch
        loss, out = self(x, y)

        y = y.squeeze().long()
        preds = torch.argmax(out, dim=1)
        acc = self.test_accuracy(preds, y)

        self.log('test_loss', loss, prog_bar=True, logger=True)
        self.log('test_acc', acc, prog_bar=True, logger=True)
        return {'loss': loss, 'acc': acc}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
        return [optimizer], [scheduler]


Now we've defined all the classes we need for the training.
The following steps will use these to train a classification model to predict future movement of USDJPY close price

In [5]:
PKL_PATH = '../data/processed/usdjpy-bar-2020-01-01-2024-12-31_processed.pkl'
SEQUENCE_LENGTH=30
HORIZON=1 # The next nth timeframe to predict
STRIDE=5 # Non-overlapping timeframe
FEATURES_COLS = ['close_return']
TARGET_COLS = ['label']

## Read data

In [6]:
import pandas as pd

In [7]:
df = pd.read_pickle(PKL_PATH)
df.head()

,timestamp,open,high,low,close,volume,time_group,close_delta,close_return,close_direction,prob_down,prob_flat,prob_up,label
1,2020-01-01 22:01:00,108.7570,108.7590,108.7495,108.7495,13300.000012,1,-0.0095,-0.000087,down,1.0,0.0,0.0,0
2,2020-01-01 22:02:00,108.7495,108.7535,108.7495,108.7535,4500.000000,1,0.0040,0.000037,up,0.0,0.0,1.0,2
3,2020-01-01 22:03:00,108.7540,108.7555,108.7535,108.7555,10490.000010,1,0.0020,0.000018,flat,0.0,1.0,0.0,1
4,2020-01-01 22:04:00,108.7575,108.7650,108.7555,108.7650,11600.000024,1,0.0095,0.000087,up,0.0,0.0,1.0,2
5,2020-01-01 22:05:00,108.7700,108.7700,108.7690,108.7700,1059.999987,1,0.0050,0.000046,up,0.0,0.0,1.0,2


## Create Datamodule

In [8]:
from utils import get_sequence_start_indices

In [9]:
## get valid indices that wont create sequences crossing time gaps
IDs = get_sequence_start_indices(
    df,
    sequence_length=SEQUENCE_LENGTH,
    horizon=HORIZON,
    stride=STRIDE,
    group_col='time_group',
)
# Initialize Data Module
dm = ForexDataModule(
    data=df,
    IDs=IDs,
    sequence_length=SEQUENCE_LENGTH,
    target=TARGET_COLS,
    features=FEATURES_COLS,
    horizon=HORIZON,
    batch_size=64,
    val_split=0.2,
    num_workers=0,
)

In [10]:
model = GRUModule(
    n_features=len(FEATURES_COLS),
    output_size=3,
    n_hidden=256,
    n_layers=3,
    dropout=0.3,
)

# Training Script

In [11]:
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import EarlyStopping
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.profilers import SimpleProfiler

### Logging

In [12]:
logger = TensorBoardLogger("lightning_logs", name="prob_gru")

In [13]:
profiler = SimpleProfiler(filename='profiler')

### Earlystopping

In [14]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=5,
    verbose=True
)

### Checkpoint

In [15]:
checkpoint_callback = ModelCheckpoint(
    filename='best_checkpoint',
    save_top_k=1,
    save_last=True,
    verbose=True,
    monitor='val_loss',
    mode='min'
)

### Trainer

In [16]:
trainer = Trainer(
    # accelerator="gpu",
    # precision='16-mixed',
    profiler=profiler,
    callbacks=[checkpoint_callback, early_stopping],
    max_epochs=200,
    logger=logger,
    gradient_clip_val=1.0
    # num_sanity_val_steps=0,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [17]:
trainer.fit(model, datamodule=dm)

You are using a CUDA device ('NVIDIA GeForce RTX 4060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params | Mode 
-------------------------------------------------------------
0 | model         | GRUModel           | 989 K  | train
1 | criterion     | CrossEntropyLoss   | 0      | train
2 | test_accuracy | MulticlassAccuracy | 0      | train
-------------------------------------------------------------
989 K     Trainable params
0         Non-trainable params
989 K     Total params
3.957     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                         | 0/? [00:00<?, ?it/s]

C:\Users\yoyo\miniconda3\envs\fxml\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
C:\Users\yoyo\miniconda3\envs\fxml\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Training: |                                                                | 0/? [00:00<?, ?it/s]

Validation: |                                                              | 0/? [00:00<?, ?it/s]

...

Epoch 41, global step 182490: 'val_loss' was not in top 1


Validation: |                                                              | 0/? [00:00<?, ?it/s]

Epoch 42, global step 186835: 'val_loss' was not in top 1


Validation: |                                                              | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 5 records. Best score: 0.029. Signaling Trainer to stop.
Epoch 43, global step 191180: 'val_loss' was not in top 1


In [18]:
trainer.test(model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\yoyo\miniconda3\envs\fxml\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Testing: |                                                                 | 0/? [00:00<?, ?it/s]

──────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
──────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9914924502372742
        test_loss          0.029169786721467972
──────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.029169786721467972, 'test_acc': 0.9914924502372742}]